In [1]:
#Install ProfileReport
!pip install -U ydata-profiling

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.5/393.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 687.8/687.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 34.7 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27081 sha256=324e73a30c9a9c809965ec68659f4b925bd6c10175f2d020fcfcc356e4680196
  Stored in directory: /root/.cache/pip/wheels/8d/55/1a/19cd535375ed1ede0c996405ebffe34b196d78e2d9545723a2
Successfully built htmlmin


In [2]:
# Imports
import pandas as pd
from ydata_profiling import ProfileReport
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from numpy import arange
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_regression, RFECV, SelectFromModel
from sklearn.metrics import r2_score
from sklearn import set_config

set_config(transform_output="pandas")

In [3]:
# Import dataset for bridge condition

bridge_dataset = pd.read_csv("/content/Last_Year_All_Field_Bridges_2.csv")
bridge_dataset

<ipython-input-3-5f18a5b05671>:3: DtypeWarning: Columns (29,84,86) have mixed types. Specify dtype option on import or set low_memory=False.
  bridge_dataset = pd.read_csv("/content/Last_Year_All_Field_Bridges_2.csv")


,1 - State Code,1 - State Name,8 - Structure Number,22 - Owner Agency,3 - County Code,3 - County Name,4 - Place Code,City - InfoBridge Place Code,City - InfoBridge Place Name,27 - Year Built,...,Number of Snowfall Days,Number of Days with Measurable Precipitation,Number of Days with Temperature Below 0?C,Prevailing Wind Direction,Time of Wetness,Total Precipitation,Metropolitan Planning Organization (MPO),U.S. Congressional District,State Senate District,State House District
0,13,Georgia,8.0302E+13,U.S. Forest Service,291,Union County,0,0,No Place Code,2012,...,NaN,141.0,76.0,158.63,4325.0,1558.15,NaN,09 - Congressional District 9,051 - State Senate District 51,008 - State House District 8
1,13,Georgia,8.0306E+13,U.S. Forest Service,241,Rabun County,0,0,No Place Code,2010,...,NaN,141.0,74.0,151.58,4453.0,1501.58,NaN,09 - Congressional District 9,050 - State Senate District 50,008 - State House District 8
2,13,Georgia,8.0305E+13,U.S. Forest Service,241,Rabun County,0,0,No Place Code,1990,...,NaN,141.0,76.0,158.63,4325.0,1558.15,NaN,09 - Congressional District 9,050 - State Senate District 50,008 - State House District 8
3,13,Georgia,5100670,State Highway Agency,51,Chatham County,0,78036,Tybee Island city,1962,...,NaN,NaN,NaN,NaN,NaN,NaN,"13198301 - Coastal Region MPO, GA",01 - Congressional District 1,001 - State Senate District 1,166 - State House District 166
4,13,Georgia,5150150,County Highway Agency,51,Chatham County,0,0,No Place Code,1975,...,NaN,NaN,NaN,NaN,NaN,NaN,"13198301 - Coastal Region MPO, GA",01 - Congressional District 1,001 - State Senate District 1,166 - State House District 166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15029,13,Georgia,12951170,State Highway Agency,129,Gordon County,0,0,No Place Code,2019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14 - Congressional District 14,052 - State Senate District 52,005 - State House District 5
15030,13,Georgia,12951150,County Highway Agency,129,Gordon County,0,0,No Place Code,2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14 - Congressional District 14,052 - State Senate District 52,005 - State House District 5
15031,13,Georgia,12951140,County Highway Agency,129,Gordon County,0,0,No Place Code,2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14 - Congressional District 14,052 - State Senate District 52,005 - State House District 5
15032,13,Georgia,4750600,County Highway Agency,47,Catoosa County,0,0,No Place Code,2020,...,NaN,NaN,NaN,NaN,NaN,NaN,47197703 - Chattanooga-Hamilton County/North G...,14 - Congressional District 14,053 - State Senate District 53,002 - State House District 2


In [4]:
# Copy (X values) from the dataset

X = bridge_dataset.copy()


In [5]:
# Target column (what we wnat to predict) pop out of the dataset

y = X.pop("CAT10 - Bridge Condition")

In [6]:
X_drop = X.drop(columns = ['58 - Deck Condition Rating',
  '59 - Superstructure Condition Rating',
  '60 - Substructure Condition Rating',
  '61 - Channel and Channel Protection Condition Rating',
  '62 - Culverts Condition Rating',
  '67 - Structural Evaluation Appraisal',
  '68 - Deck Geometry Appraisal',
  '69 - Underclearance Appraisal Vertical and Horizontal',
  '71 - Waterway Adequacy Appraisal'])

In [7]:
X_numerical = X[['58 - Deck Condition Rating',
  '59 - Superstructure Condition Rating',
  '60 - Substructure Condition Rating',
  '61 - Channel and Channel Protection Condition Rating',
  '62 - Culverts Condition Rating',
  '67 - Structural Evaluation Appraisal',
  '68 - Deck Geometry Appraisal',
  '69 - Underclearance Appraisal Vertical and Horizontal',
  '71 - Waterway Adequacy Appraisal'
  ]].apply(pd.to_numeric, errors='coerce')

In [8]:
X_new= pd.concat([X_drop, X_numerical], axis=1)

In [9]:
X_new["CAT10 - Bridge Condition"] = y

In [10]:
X_without_code = X_new.drop("CAT23 - Condition Code", axis=1)

In [11]:
new_modalar_df = X_without_code.sample(n=500)

In [12]:
new_modalar_df.to_csv("new_big_bridge_data_df.csv")

In [26]:
!pip install -q streamlit
!pip install folium
!pip install streamlit-folium
!pip install streamlit pydeck

In [29]:
# code necessary for Colab only

import os
import time

from IPython.display import display, HTML
def tunnel_prep():
    for f in ('cloudflared-linux-amd64', 'logs.txt', 'nohup.out'):
        try:
            os.remove(f'/content/{f}')
            print(f"Deleted {f}")
        except FileNotFoundError:
            continue

    !wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -q
    !chmod +x cloudflared-linux-amd64
    !nohup /content/cloudflared-linux-amd64 tunnel --url http://localhost:8501 &
    url = ""
    while not url:
        time.sleep(1)
        result = !grep -o 'https://.*\.trycloudflare.com' nohup.out | head -n 1
        if result:
            url = result[0]
    return display(HTML(f'Your tunnel URL <a href="{url}" target="_blank">{url}</a>'))

In [79]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from numpy import arange
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_regression, RFECV, SelectFromModel
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from PIL import Image
import pydeck
import folium
from folium import plugins
from streamlit_folium import folium_static
from streamlit_folium import st_folium
import pydeck as pdk
import numpy as np
from folium.plugins import MarkerCluster
from branca.colormap import linear
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

st.set_page_config(
    page_title="Ex-stream-ly Cool App",
    layout="wide",
    initial_sidebar_state="collapsed"
)
# Add a title
st.title("Bridge condition predictor app!")
# Add a comment(explanation)
st.info("This app is predicting the condition of bridges")

image = Image.open("/content/bridge_2.jpg")
st.image(image, caption = "Dames Point Bridge , Jacksonville")

tab1, tab2, tab3, tab4 = st.tabs(["Data", "Data visualization", "Bridge Condition predictor", "Input new bridge"])
with tab1:
    col1, col2 = st.columns([4,1])
    with col1:
        st.write("**Raw Data**")
        bridge_info = pd.read_csv("/content/new_big_bridge_data_df.csv")
        st.write(bridge_info.sample(n=20, ignore_index=True))
    with col2:
        st.write("**Target value**")
        X = bridge_info.copy()
        y = X.pop("CAT10 - Bridge Condition")
        st.write(y.sample(n=20, ignore_index=True))
# map with bridges add markers


map = folium.Map(location=[33.247875, -83.441162], zoom_start=6, tiles="cartodb positron")

category_colors = {
    "Good" : "green",
    "Fair" : "blue",
    "Poor" : "red"
}
for index, row in bridge_info.iterrows():
    cat_color = category_colors.get(row['CAT10 - Bridge Condition'], "gray")
    # Popup content will show column names and corresponding values
    popup_content = f"<strong>Structure Length(ft.):</strong> {row['49 - Structure Length (ft.)']}<br>"
    popup_content += f"<strong>Bridge Age(yr):</strong> {row['Bridge Age (yr)']}<br>"
    popup_content += f"<strong>Deck Area(sq. ft.):</strong> {row['CAT29 - Deck Area (sq. ft.)']}<br>"
    popup=folium.Popup(popup_content, max_width=300)
    folium.CircleMarker(
        location = [row["latitude"], row["longitude"]],
        radius=6,
        color=cat_color,
        popup=popup,
        icon=folium.Icon(color=cat_color),
        weight=1,
        fill_opacity=0.6,
        opacity=1,
        fill=True,
    ).add_to(map)

with tab2:
    col1, col2 = st.columns([2,2])
    with col1:
        # Create a histogram using seaborn
        sns.set(style="whitegrid")
        plt.figure(figsize=(10, 10))
        sns.histplot(bridge_info,
                    x=st.selectbox("What do you want in x axis",("Bridge Age (yr)","49 - Structure Length (ft.)","Computed - Average Daily Truck Traffic (Volume)","29 - Average Daily Traffic","58 - Deck Condition Rating" ),
                    placeholder="Select data visualization"), hue="CAT10 - Bridge Condition",
                    stat="count",
                    common_norm=False,
                    )
        # Show the plot in Streamlit
        st.pyplot(plt)
    with col2:
        st.scatter_chart(data=bridge_info,
                         y=st.selectbox("What do you want in y axis",("61 - Channel and Channel Protection Condition Rating","58 - Deck Condition Rating", "60 - Substructure Condition Rating", "59 - Superstructure Condition Rating"),placeholder="Select data visualization"),
                         x=st.selectbox("What do you want in x axis",("Bridge Age (yr)","Total Precipitation","29 - Average Daily Traffic","Minimum Temperature"),placeholder="Select data visualization"),
                         color="CAT10 - Bridge Condition")




# Extract target and features
X = bridge_info.drop(columns=['CAT10 - Bridge Condition'])  # Remove target column
y = bridge_info['CAT10 - Bridge Condition']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include=['object']).columns
numerical_cols = X.select_dtypes(exclude=['object']).columns

# Create the preprocessing pipelines for both numerical and categorical columns
numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant')),
])

categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='infrequent_if_exist',min_frequency=0.02,sparse_output=False))
])

# Create the column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_cols),
        ('cat', categorical_pipeline, categorical_cols)
    ]
)

# Build the model pipeline with a RandomForestClassifier
model = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])
# Hyperparameter grid for GridSearchCV
param_grid = {
    'classifier__n_estimators': [50, 100,10],
    'classifier__ccp_alpha': arange(0, 0.1, 0.02)
}

grid_search = GridSearchCV(model,
                           param_grid,
                           cv=5,
                           n_jobs=-1,
                           verbose=1)

# Train the model
grid_search.fit(X_train, y_train)

# Streamlit form for new bridge data input

with st.sidebar.form(key ='Form1'):
    st.header('Input New Bridge Data')

    # Define input fields
    state = st.selectbox("State", ["Alabama", "Georgia", "Florida", "Texas", "California"],index=None, placeholder="Select state")
    st.caption(f"You selected: {state}")
    latitude = st.number_input("Latitude", -90.0, 90.0, 0.0)
    st.caption(f"Latitude: {latitude}")
    longitude = st.number_input("Longitude", -180.0, 180.0, 0.0)
    st.write(f"Longitude: {longitude}")
    structure_length = st.slider("Structure Length (ft.)", 0, 10000, 1000)
    st.caption(f"You selected: {structure_length}")
    bridge_age = st.slider("Bridge Age (years)", 0, 100, 20)
    st.caption(f"You selected: {bridge_age}")
    deck_area = st.slider("Deck Area (sq. ft.):", 200 , 100000, 100)
    st.caption(f"You selected: {deck_area}")
    deck_condition_rating = st.slider("58 - Deck Condition Rating:", 0 , 10, 1)
    st.caption(f"You selected: {deck_condition_rating}")
    superstructure_cond_rating = st.slider("59 - Superstructure Condition Rating:", 0 , 10, 1)
    st.caption(f"You selected: {superstructure_cond_rating}")
    substructure_cond_rating = st.slider("60 - Substructure Condition Rating:", 0 , 10, 1)
    st.caption(f"You selected: {substructure_cond_rating}")
    channel_cond_rating = st.slider("61 - Channel and Channel Protection Condition Rating:", 0 , 10, 1)
    st.caption(f"You selected: {channel_cond_rating}")
    culverts_cond_rating = st.slider("62 - Culverts Condition Rating:", 0 , 10, 1)
    st.caption(f"You selected: {culverts_cond_rating}")

    data = {"1 - State Name" : state,
            "49 - Structure Length (ft.)" : structure_length,
            'Bridge Age (yr)' : bridge_age,
            'CAT29 - Deck Area (sq. ft.)' : deck_area,
            '58 - Deck Condition Rating' : deck_condition_rating,
            '59 - Superstructure Condition Rating' : superstructure_cond_rating,
            '60 - Substructure Condition Rating' : substructure_cond_rating,
            '61 - Channel and Channel Protection Condition Rating' : channel_cond_rating,
            '62 - Culverts Condition Rating' : culverts_cond_rating,
            "latitude" : latitude,
            "longitude" : longitude
    }
    input_df = pd.DataFrame(data, index=[0])
    # List of columns in df1 that are not in df2
    cols_to_add = [col for col in bridge_info.columns if col not in input_df.columns]

    # Add the new columns to df2 with None values
    for col in cols_to_add:
        input_df[col] = None
    input_df.drop(columns =['CAT10 - Bridge Condition'])
    submitted1 = st.form_submit_button(label = 'Add bridge info')
# Initialize session state to store the merged data and map entries
if 'map_entries' not in st.session_state:
    st.session_state.map_entries = []
with tab3:
    # Predict bridge condition
    st.header("Prediction")

    if st.button('Predict'):
        prediction = grid_search.predict(input_df)
        st.metric("Predicted condition of the new inputed bridge", prediction[0])
        # Create the map visualization
        # Merge new data with the original dataset
        new_row = input_df.copy()
        new_row['CAT10 - Bridge Condition'] = prediction
        # Append the new data entry as a dictionary (1D format) to the session state
        st.session_state.map_entries.append(new_row.iloc[0].to_dict())# Convert the row to a dictionary
        df_new = pd.DataFrame(st.session_state.map_entries)
        st.write(df_new)
        full_data = pd.concat([df_new, bridge_info],axis=0, ignore_index=True)

        # Add markers for each entry in the map_entries list
        for entry in st.session_state.map_entries:
            # Popup content will show column names and corresponding values
            popup_content = f"<strong>Structure Length(ft.):</strong> {row['49 - Structure Length (ft.)']}<br>"
            popup_content += f"<strong>Bridge Age(yr):</strong> {row['Bridge Age (yr)']}<br>"
            popup_content += f"<strong>Deck Area(sq. ft.):</strong> {row['CAT29 - Deck Area (sq. ft.)']}<br>"
            popup=folium.Popup(popup_content, max_width=300)
            # Marker based on the predicted condition
            condition = prediction[0]  # The predicted condition
            marker_color = {'Good': 'green', 'Fair': 'blue', 'Poor': 'red'}.get(entry['CAT10 - Bridge Condition'], 'gray')
            folium.CircleMarker(
                location=[entry['latitude'], entry['longitude']],
                popup=popup,
                icon=folium.Icon(color=marker_color),
                weight=1,
                color=marker_color,
                fill_opacity=0.6,
                opacity=1,
                fill=True,
                radius=6
            ).add_to(map)

        folium_static(map)

    else:
        print("Not predicted yet")

with tab4:
    col1, col2 = st.columns([2,2])
    with col1:
        st.write("**Input bridge data**")
        input_df
    with col2:
        st.write("**Merge all data**")
        # Ensure st.session_state.map_entries is a DataFrame
        if isinstance(st.session_state.map_entries, list):
            # Convert the list of dictionaries into a DataFrame
            map_entries_df = pd.DataFrame(st.session_state.map_entries)
        st.write(pd.concat([map_entries_df, bridge_info], ignore_index=True))


Overwriting app.py


In [31]:
tunnel_prep()

Deleted cloudflared-linux-amd64
Deleted nohup.out
nohup: appending output to 'nohup.out'


In [32]:
!streamlit run app.py &>/content/logs.txt &